In [595]:
import json
import re

with open('cv_entries.json', 'r') as file:
    data = json.load(file)


latex_greek_chars = {
    "α": r"$\alpha$",
    "β": r"$\beta$",
    "γ": r"$\gamma$",
    "δ": r"$\delta$",
    "ε": r"$\epsilon$",
    "θ": r"$\theta$",
    "λ": r"$\lambda$",
    "μ": r"$\mu$",
    "π": r"$\pi$",
    "ρ": r"$\rho$",
    "σ": r"$\sigma$",
    "τ": r"$\tau$",
    "φ": r"$\phi$",
    "χ": r"$\chi$",
    "ψ": r"$\psi$",
    "ω": r"$\omega$",
    "Δ": r"$\Delta$",
    "Γ": r"$\Gamma$",
    "Σ": r"$\Sigma$",
    "Φ": r"$\Phi$",
    "Ψ": r"$\Psi$",
    "Ω": r"$\Omega$",
    "√sNN" : r"$\sqrt{s_{NN}}$"
}

def translate_to_tex(input):
    for raw in latex_greek_chars:
        input = input.replace(raw, latex_greek_chars[raw])
    return input

def fix_units_spacing(text):
    # Matches a number (int or float) followed by optional space(s), then units (letters, %, °, etc.)
    pattern = r'(\d+(?:\.\d+)?)(\s*)([a-zA-Z%°]+)'
    # Replace with number + ~ + units (non-breaking space)
    fixed_text = re.sub(pattern, r'\1~\3', text)
    return fixed_text

def to_sentence(text):
    text = text[0].upper() + text[1:]
    return text

In [596]:
full_tex = r"""
\documentclass[11pt,a4paper]{article}
\usepackage{array}
\usepackage[margin=1in]{geometry}
\usepackage{enumitem}
\usepackage{hyperref}
\usepackage{graphicx}
\usepackage{titlesec}
\usepackage{parskip}
\usepackage[dvipsnames]{xcolor}
\usepackage{multirow}
\usepackage{amsmath}
\usepackage{MnSymbol}
\usepackage{wasysym}


\setlength{\parindent}{0pt}
\titleformat{\section}{\Large\bfseries}{}{0em}{}
\titleformat{\subsection}{\normalsize\bfseries}{}{0em}{}

\begin{document}
\renewcommand{\baselinestretch}{1.0}
"""

##### Header Section

In [597]:

full_tex += r"""
\begin{table}[htb!]
    \begin{tabular}{p{0.7\textwidth} p{0.3\textwidth}}
        \Large
"""
full_tex += r"\centering{\textbf{{" + data['profile']['name'] + r"}}" + "\\\\\n" + r"\vspace{-5pt}"
full_tex += r"""
\vspace{-5pt}
\par\noindent\rule{0.9\linewidth}{0.2pt}
"""
full_tex += r"\large{{" + "-".join(data['profile']['brief'].split('|')) + r"}}}\\"


full_tex += r"""
         & \multirow{5}{*}{\reflectbox{\includegraphics[scale=0.04]{images/profile_image.jpeg}}}\\
        \normalsize
"""


full_tex += r"\begin{tabular}{>{\raggedleft\arraybackslash}p{0.20\linewidth} p{0.7\linewidth}}" + "\n"
full_tex += r"\textbf{Affiliations}: & " + f"{data['profile']['affiliations'][0]['name']}, {data['profile']['affiliations'][0]['location']}" + " \\\\\n"
for aff in data['profile']['affiliations'][1:]:
    full_tex += " & " f"{aff['name']}, {aff['location']}\\\\\n"
full_tex += r"\textbf{Address}: & " + data['profile']['address'] + "\\\\\n"
emails = [f"\\href{{mailto:{e}}}{{{e}}}" for e in data['profile']['email']]
full_tex += r"\textbf{E-mail}: & " + ", ".join(emails) + r"\\\n"
full_tex += r"\textbf{Born}: & " + data['profile']['birth'] + "\\\\\n"
full_tex += r"\textbf{Homepage}: & \href{" + f"{data['profile']['homepage']}}}{{{data['profile']['homepage']}}}"  + "\\\\\n"

full_tex += "\\end{tabular}\n"


full_tex += r"""
    \end{tabular}
\end{table}
"""

##### Skills Summary

In [598]:
full_tex += r"""
\vspace*{0.3cm}
\noindent  \textbf{\Large \color{Blue} Skill Summary:}
\vspace{-15pt}
\par\noindent\rule{\textwidth}{0.4pt}
"""

full_tex += "\tLenguages: " + ", ".join(data['skills_summary']['languages']) + "\\\\\n"
full_tex += "\tProgramming: " + ", ".join(data['skills_summary']['programming']) + "\\\\\n"
full_tex += "\tAdditional tools: " + ", ".join(data['skills_summary']['additional_tools']) + "\\\\\n"

full_tex += "\\begin{tabular}{ll}\n"
full_tex += "Soft skills: & " + data['skills_summary']['soft_skills'][0] + " \\\\\n"

for skill in data['skills_summary']['soft_skills'][1:]:
    full_tex += " & " + skill + " \\\\\n"

full_tex += "\\end{tabular}\n"



full_tex += "\n\n"

##### Profesional Experience

In [599]:
full_tex += r"""
\vspace*{0.3cm}
\noindent  \textbf{\Large \color{Blue} Professional experience:}
\vspace{-15pt}
\par\noindent\rule{\textwidth}{0.4pt}
\begin{itemize}
"""


for entry in data['professional_experience']:
    tex  = "\t" + r"\item[\scalebox{0.5}{$\blacksquare$}] \textbf{"
    tex += "\t" + entry["date_start"] +" - "+ entry["date_end"] + ":} " + entry["role"] + ", " + entry["institution"] + " - " + entry['department'] + r".\\"+ "\n"
    tex += "\t" + entry['details'] + "\n"
    full_tex += tex

full_tex += r"\end{itemize}"
full_tex += "\n\n"


##### Education

In [600]:
full_tex += r"""
\vspace*{0.3cm}
\noindent \textbf{\Large \color{Blue} Education:}
\vspace{-15pt}
\par\noindent\rule{\textwidth}{0.4pt}
\begin{itemize}
"""
for entry in data['education']:
    tex  = "\t" + r"\item[\scalebox{0.5}{$\blacksquare$}] \textbf{"
    tex += entry["date_start"] +" - "+ entry["date_end"] + ":} " + entry["degree"] + ", " + entry["institution"] + r".\\" + "\n"
    tex += "\t" + r"\textbf{Thesis}: \textit{" + fix_units_spacing(entry["details"]["thesis"]) + r"}\\"+ "\n"
    tex += "\t" + r"\textbf{Supervisor}: " + entry["details"]["supervisor"] + "\n"
    full_tex += tex

full_tex += r"\end{itemize}"
full_tex += "\n\n"

##### Summer schools and internships

In [601]:
full_tex += r"""
\noindent \textbf{Summer schools and internships:}
\begin{itemize}
"""

for entry in data['summer_schools_internships']:
    tex  = "\t" + r"\item[\scalebox{0.5}{$\blacksquare$}]\textbf{"
    tex += entry["date_start"] +" - "+ entry["date_end"] + ":} " + entry["program"] + ", " + entry["location"] + r".\\"+ "\n"
    tex += "\t" + r"\textbf{Work Topic}: \textit{" + fix_units_spacing(entry["details"]["work_report"]) + r"}\\"+ "\n"
    tex += "\t" + r"\textbf{Supervisor}: " + entry["details"]["supervisor"] + "\n"
    full_tex += tex

full_tex += r"\end{itemize}"
full_tex += "\n\n"

##### Publications

In [602]:
full_tex += r"""
\vspace*{0.3cm}
\noindent  \textbf{\Large \color{Blue} Publications:}
\vspace{-15pt}
\par\noindent\rule{\textwidth}{0.4pt}
\begin{itemize}
"""

for entry in data['publications']['journal']:
    tex  = "\t" + r"\item[\scalebox{0.5}{$\blacksquare$}]" + f" {entry['authors']}, \\textit" + '{' + f"{fix_units_spacing(translate_to_tex(entry['title']))}." + "} "
    tex += f"{entry['journal']}"
    if len(entry['doi']) != 0:
        tex += f", {entry['year']} \\\\\n\t{entry['doi']}\n"
    else:
        tex += f"\\\\\n\t{entry['details']}\n"
        print(f"No DOI available for {entry}")
    full_tex += tex

for entry in data['publications']['conference_proceedings']:
    tex  = "\t" + r"\item[\scalebox{0.5}{$\blacksquare$}]" + f" {entry['authors']}, \\textit" + '{' + f"{fix_units_spacing(translate_to_tex(entry['title']))}." + '}' + "\\\\\n"
    tex += f"\tProceedings: {entry['conference']}, {entry['date']}, {entry['location']}\\\\\n"
    try:
        doi = entry['doi']
        tex += f"\t{doi}\n"
    except:
        tex += f"\t{entry['details']}\n"
        print(f"No DOI available for {entry}")
    full_tex += tex

for entry in data['publications']['technical_reports']:
    tex  = "\t" + r"\item[\scalebox{0.5}{$\blacksquare$}]" + f" {entry['authors']}, \\textit" + '{' + f"{fix_units_spacing(translate_to_tex(entry['title']))}." + "} "
    tex += f"{entry['journal']}"
    if len(entry['doi']) != 0:
        tex += f", {entry['year']} \\\\\n\t{entry['doi']}\n"
    else:
        tex += f"\\\\\n\t{entry['details']}\n"
        print(f"No DOI available for {entry}")
    full_tex += tex

full_tex += r"\end{itemize}"
full_tex += "\n\n"

No DOI available for {'title': 'Performance of the prototype Silicon Tracking System of the CBM experiment tested with heavy-ion beams at SIS18', 'authors': 'D. Ramirez et al. (CBM Collaboration)', 'journal': 'Nuclear Inst. and Methods in Physics Research, A', 'year': '', 'vol': '', 'pages': '', 'doi': '', 'details': 'Submission Being Processed (June 2025)'}
No DOI available for {'title': 'Performance of hit, track, and vertex reconstruction of the Silicon Tracking System of the CBM experiment', 'authors': 'D. Ramirez', 'conference': 'FAIR next generation scientists - 8th Edition Workshop (FAIRness2024)', 'location': 'Seget, Croatia', 'date': 'Sept 23-27, 2024', 'details': ' (On Editorial phase)'}


##### Conferences

In [603]:
full_tex += r"""
\vspace*{0.3cm}
\noindent  \textbf{\Large \color{Blue} Conference Participation:}
\vspace{-15pt}
\par\noindent\rule{\textwidth}{0.4pt}
\begin{itemize}
"""

for entry in data['conference_participation']:
    tex  = "\t" + r"\item[\scalebox{0.5}{$\blacksquare$}]" + f" {entry['name']}, {entry['date']}, {entry['location']}" + r".\\" + '\n'
    if len(entry['contribution']) != 0:
        tex += "\t" + r"Contribution: \textit{" + fix_units_spacing(translate_to_tex(entry['contribution'])) + r"}" + "\n"
    else:
        tex = tex.replace(r"\\", "")
    full_tex += tex

full_tex += r"\end{itemize}"
full_tex += "\n\n"


##### Awards

In [604]:
full_tex += r"""
\vspace*{0.3cm}
\noindent  \textbf{\Large \color{Blue} Awards:}
\vspace{-15pt}
\par\noindent\rule{\textwidth}{0.4pt}
\begin{itemize}
"""

for entry in data['awards']:
    tex  = "\t" + r"\item[\scalebox{0.5}{$\blacksquare$}]" + f" {entry}.\\\\\n"
    full_tex += tex

full_tex += r"\end{itemize}"
full_tex += "\n\n"

##### Wrap up

In [605]:
full_tex += "\n\n" + r"\end{document}"

full_tex = translate_to_tex(full_tex)


with open("cv.tex", "w") as f:
    f.writelines(full_tex)